In [1]:
latest_gameweek = 6

In [2]:
import pandas as pd
import numpy as np
from pathlib import Path

import gradio as gr
from scipy.ndimage import gaussian_filter

import plotly.graph_objects as go
import seaborn as sns
sns.set_style("darkgrid")


# Data

In [3]:
# POSITION MAPPING
position_dict = {1:'GK', 2:'DEF', 3:'MID', 4:'FWD'}

In [ ]:
filepath = Path(f'../data/predictions/gameweek{latest_gameweek}.csv')
projections = pd.read_csv(filepath, index_col=0)
projections['position'] = projections['element_type'].map(position_dict)
display(projections.head())
display(projections.shape)

In [ ]:
if latest_gameweek>0:
    filepath = Path('../data/fpl_df.csv')
elif latest_gameweek==0:
    filepath = Path('../data/fpl_df_preseason.csv')
fpl_df = pd.read_csv(filepath, index_col=0, low_memory=False)
fpl_df = fpl_df[fpl_df.season=='24-25']
display(fpl_df.head())
display(fpl_df.shape)

In [ ]:
[col for col in fpl_df.columns if 'oints' in col]

In [ ]:
df = fpl_df.groupby('name').last().reset_index()[['name', 'team_name', 'element_type', 'now_cost', 
                            'gameweek_minutes_ewm_20', 'points_per_game', 'total_points', 
                            'gameweek_xPoints_ewm_5', 'gameweek_xPoints_ewm_10', 'gameweek_xPoints_ewm_20', 'gameweek_xPoints_ewm_40']]
df['games_played'] = np.round(np.where(df['points_per_game']!=0, df['total_points'] / df['points_per_game'], 0),0)
df['price'] = df['now_cost'] / 10.0
df['value'] = df['gameweek_xPoints_ewm_20'] / df['price']
df['value_points'] = np.sqrt( df['gameweek_xPoints_ewm_20'] *  df['value'])

# EXPECTED POINTS
expected_points_next_10gw = (projections[projections.gameweek.isin( np.arange(latest_gameweek+1, latest_gameweek+11, 1) )]
 .groupby('name')
 .sum()
 )[['expected_points']].reset_index().rename(columns={'expected_points':'expected_points_next_10_GW'})

expected_points_next_5gw = (projections[projections.gameweek.isin( np.arange(latest_gameweek+1, latest_gameweek+6, 1) )]
 .groupby('name')
 .sum()
 )[['expected_points']].reset_index().rename(columns={'expected_points':'expected_points_next_5_GW'})

df = df.merge(expected_points_next_10gw, on='name', how='left')
df = df.merge(expected_points_next_5gw, on='name', how='left')

# POSITION MAPPING
df['position'] = df['element_type'].map(position_dict)

# SEASON TOTALS AND MEANS (XPOINTS AND POINTS)

season_total_xPoints = \
    (fpl_df
     .groupby('name')[['gameweek_xPoints']]
     .sum()
     .rename(columns={'gameweek_xPoints':'season_total_xpoints'})
     .reset_index())

season_total_points = \
    (fpl_df
     .groupby('name')[['event_points']]
     .sum()
     .rename(columns={'event_points':'season_total_points'})
     .reset_index())

season_mean_xPoints = \
    (fpl_df
     .groupby('name')[['gameweek_xPoints']]
     .mean()
     .rename(columns={'gameweek_xPoints':'season_mean_xpoints'})
     .reset_index())

season_mean_points = \
    (fpl_df
     .groupby('name')[['event_points']]
     .mean()
     .rename(columns={'event_points':'season_mean_points'})
     .reset_index())

df = df.merge(season_total_xPoints, on='name', how='left')
df = df.merge(season_total_points, on='name', how='left')
df = df.merge(season_mean_xPoints, on='name', how='left')
df = df.merge(season_mean_points, on='name', how='left')

display(df)

# Gradio functions

In [8]:
def plot_points_and_value(positions, teams, show_names, max_price, x_axis_feature, y_axis_feature):
    
    fig = go.Figure()

    # aux df for manipulation
    df_out = df[df.position.isin(positions)].copy()
    # choose only given teams
    if "Select All" not in teams:
        df_out = df_out[df_out.team_name.isin(teams)]

    # drop players above max_price
    df_out = df_out[df_out.price<=max_price]

    # CREATE FIGURE
    fig.add_trace(
            go.Scatter(
                x=df_out[x_axis_feature],
                y=df_out[y_axis_feature],
                mode="markers+text",
                hovertext=df_out['name'].values,
                showlegend=False,
                ),
        )

    # add player names as visible
    if show_names:
        fig.update_traces(
            text = df_out['name'].values,
            textposition='top center',
            )

    # styling
    fig.update_layout(
        #title="",
        template='plotly_dark',
        xaxis_title=x_axis_feature,
        yaxis_title=y_axis_feature,
        #showlegend=True
    )

    return fig

In [9]:
def top_weekly_projections(positions, teams, gameweek):
    
    fig = go.Figure()

    # aux df for manipulation
    projections_out = projections[projections.position.isin(positions)].copy()
    # choose only given teams
    if "Select All" not in teams:
        projections_out = projections_out[projections_out.team_name.isin(teams)]
    # choose gameweek
    projections_out = projections_out[projections_out.gameweek==int(gameweek)]

    top_20 = projections_out.groupby('name').sum().sort_values(by='expected_points').reset_index().tail(20)

    # CREATE FIGURE
    fig.add_trace(
        go.Bar(
            x=top_20.expected_points,
            #y=np.arange(top_20.shape[0],0,-1), 
            y=top_20.name, 
            text=np.round(top_20.expected_points,2),
            textposition='outside',
            width=0.75,
            orientation='h'
        ),
    ) 

    # styling
    fig.update_layout(
        #title="",
        template='plotly_dark',
        xaxis_title='gameweek expected points',
        #yaxis_title='value',
        #showlegend=True
    )

    return fig

In [10]:
def player_points(players):

    marker_colors = ['red', 'blue']
    colors = ['rgba(255, 0, 0, 0.5)','rgba(0, 0, 255, 0.5)']
    my_df = pd.DataFrame()

    fig = go.Figure()
    for count, player in enumerate(players):

        aux = df.loc[df['name']==player, ['name', 'team_name', 'position', 'price', 
                            'points_per_game', 'total_points', 
                            'gameweek_xPoints_ewm_5', 'gameweek_xPoints_ewm_40', ]].copy()

        my_past_data = fpl_df[fpl_df['name']==player].sort_values(by='gameweek')
        my_projections = projections[projections['name']==player].sort_values(by='gameweek')
        
        x_past = list(np.unique(my_past_data['gameweek']))
        x_future = list(np.unique(my_projections['gameweek']))
        my_x = x_past + x_future

        y_past = list(my_past_data.groupby('gameweek').sum()['gameweek_xPoints'])
        y_future = list(my_projections.groupby('gameweek').sum()['expected_points'])
        my_y = y_past + y_future
        my_y_filtered = gaussian_filter(y_past + y_future, sigma=2, mode='nearest')

        new_cols = [f'xPoints_gameweek_{i}' for i in x_future]
        aux[new_cols] = y_future
        my_df = pd.concat([my_df, aux])

        fig.add_trace(
            go.Scatter(
                x=my_x,
                y=my_y_filtered,
                mode="markers+lines",
                marker=dict(color=marker_colors[count]),  
                line=dict(color=marker_colors[count], width=3),  
                fill='tozeroy',    
                fillcolor=colors[count],   
                name=player,            
                showlegend=True,
                ),
        )

        if len(players)==1:
            fig.add_trace(
                go.Scatter(
                    x=my_x,
                    y=my_y,
                    mode="markers",
                    marker=dict(color='white'),
                    name=player,            
                    showlegend=False,
                    ),
            )

    fig.add_vline(x=latest_gameweek+0.5,)

    fig.update_layout(
        #title="",
        template='plotly_dark',
        xaxis_title="gameweek",
        yaxis_title='expected points',
        #showlegend=True
    )

    return fig, my_df

# Demo

In [11]:
position_list = ['GK', 'DEF', 'MID', 'FWD']
team_name_list = ["Select All"]
team_name_list += list(np.sort(df.team_name.unique()))
gameweek_list = [str(x) for x in np.arange(latest_gameweek+1, latest_gameweek+11,)]
features = ["gameweek_xPoints_ewm_5", "gameweek_xPoints_ewm_20", 'expected_points_next_10_GW', 'value', 
            'season_total_xpoints', 'season_total_points', 'season_mean_xpoints', 'season_mean_points']
x_axis_feature = features
y_axis_feature = features
minimum_price = df['price'].min()
maximum_price = df['price'].max()

In [12]:
if latest_gameweek>0:
    scatter_demo = gr.Interface(
        plot_points_and_value,
        [
            gr.CheckboxGroup(position_list, label="POSITION", value=position_list),
            gr.Dropdown(team_name_list, label="TEAM", multiselect=True, value="Select All"),
            gr.Checkbox('Show player names', value=False),
            gr.Slider(minimum_price, maximum_price, value=maximum_price, info='Choose maximum allowed player value.'),
            gr.Dropdown(x_axis_feature, label="x-axis", value='gameweek_xPoints_ewm_20'),
            gr.Dropdown(y_axis_feature, label="y-axis", value = 'value'),
        ],
        gr.Plot(),
    )

In [13]:
weekly_top_players_demo = gr.Interface(
    top_weekly_projections,
    [
        gr.CheckboxGroup(position_list, label="POSITION", value=position_list),
        gr.Dropdown(team_name_list, label="TEAM", multiselect=True, value="Select All"),
        gr.Dropdown(gameweek_list, label="GAMEWEEK", value=str(latest_gameweek+1)),
    ],
    gr.Plot(),
)

In [14]:
player_name_list = list(projections.name.unique())

In [15]:
# player_points_demo = gr.Interface(
#     fn=player_points,
#     inputs=[
#         gr.Dropdown(player_name_list, value='Erling Haaland', multiselect=True, max_choices=2, label='Choose 1 or 2 players')
#     ],
#     outputs=[gr.Plot(), gr.DataFrame()]
# )

In [16]:
with gr.Blocks() as player_points_demo:
    with gr.Column():
        choose_players = gr.Dropdown(
            player_name_list, 
            value='Erling Haaland', 
            multiselect=True, 
            max_choices=2, 
            label='Choose 1 or 2 players'
            )
        submit_button = gr.Button("Submit")
        player_plot = gr.Plot()
        player_data = gr.DataFrame()
    
    submit_button.click(fn=player_points, inputs=choose_players, outputs=[player_plot, player_data])

In [ ]:
if latest_gameweek>0:
    full_demo = gr.TabbedInterface(
        [scatter_demo, weekly_top_players_demo, player_points_demo],
        ['Scatter plots', 'Gameweek projections top 20', 'Player xPoints and projections']
    ).launch()
elif latest_gameweek==0:
    full_demo = gr.TabbedInterface(
        [weekly_top_players_demo, player_points_demo],
        ['Gameweek projections top 20', 'Player xPoints and projections']
    ).launch()
else:
    print("Choose valid 'latest_gameweek'!")

# DEV STUFF